# 0. Imports

In [2]:
# Basic imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

import math

import datasus_dbc # Used to decompress the DBC file from DATASUS
import dbfread # Used to read the decompressed DBF

from IPython.core.display import HTML
from IPython.display import Image

In [3]:
# Set the option to display all columns in the DataFrame
pd.set_option('display.max_columns', None)

# 1. Class RT_APAC

In [ ]:
class RT_APAC:
      
    def concat_all_data(path_files, state): # Final dictionary to hold the results  

        # Create a full data DataFrame to hold all data
        full_data = pd.DataFrame()            
        
        # Path to the directory containing the DBC files
        path_files = os.path.join(path_files, state)  # Adjust the path as needed

        # Check all paths in the directory
        for path in os.listdir(path_files):            

            # Create an empty DataFrame to hold all data
            all_data = pd.DataFrame()

            # Print the path being processed
            print("Processing path:", path, end=' from -> ')            

            # Construct the full path to the directory            
            path_destiny = os.path.join(path_files, path)
            print(path_destiny)            
        
            # Check all files in the path
            try:
                for file_to_read in os.listdir(path_destiny):                
                
                    # Construct the full path to the file
                    file_to_read = os.path.join(path_destiny, file_to_read)
                    # print("Reading file:", file_to_read)   
                
                    # Decompress the DBC file to a temporary DBF file
                    datasus_dbc.decompress(file_to_read, "output.dbf")

                    # Read the decompressed DBF file using dbfread
                    dbf = dbfread.DBF('output.dbf', encoding='latin1')
                    #df = pd.DataFrame(iter(dbf))

                    # Now you can work with the DataFrame
                    df = pd.DataFrame(iter(dbf))  # Convert to DataFrame

                    for col in df.columns:
                        # Convert columns to string type
                        df[col] = df[col].astype(str)                

                    # Concatenate the DataFrame to the main DataFrame                
                    all_data = pd.concat([all_data, df], ignore_index=True)            
                
                    # Construct the full path to save the CSV file
                    csv_destiny = os.path.join(path_files, f"{path}.csv") 
            except NotADirectoryError:
                print(f"Error: {path_destiny} is not a directory or does not exist.")
                pass               
        
            # Concatenate the all_data DataFrame to the full_data DataFrame
            full_data = pd.concat([full_data, all_data], ignore_index=True)
        

        # Save the full_data DataFrame to a CSV file
        csv_destiny = os.path.join(path_files, f"{state}.csv")
        full_data.to_csv(csv_destiny, index=False)

In [23]:
data = RT_APAC.concat_all_data("C:\\Users\\175 MX\\Documents\\Gustavo\\datasus\\data_rt_states\\states\\", "sc")

Processing path: 08 from -> C:\Users\175 MX\Documents\Gustavo\datasus\data_rt_states\states\sc\08
Reading file: C:\Users\175 MX\Documents\Gustavo\datasus\data_rt_states\states\sc\08\ARSC0801.dbc
Reading file: C:\Users\175 MX\Documents\Gustavo\datasus\data_rt_states\states\sc\08\ARSC0802.dbc
Reading file: C:\Users\175 MX\Documents\Gustavo\datasus\data_rt_states\states\sc\08\ARSC0803.dbc
Reading file: C:\Users\175 MX\Documents\Gustavo\datasus\data_rt_states\states\sc\08\ARSC0804.dbc
Reading file: C:\Users\175 MX\Documents\Gustavo\datasus\data_rt_states\states\sc\08\ARSC0805.dbc
Reading file: C:\Users\175 MX\Documents\Gustavo\datasus\data_rt_states\states\sc\08\ARSC0806.dbc
Reading file: C:\Users\175 MX\Documents\Gustavo\datasus\data_rt_states\states\sc\08\ARSC0807.dbc
Reading file: C:\Users\175 MX\Documents\Gustavo\datasus\data_rt_states\states\sc\08\ARSC0808.dbc
Reading file: C:\Users\175 MX\Documents\Gustavo\datasus\data_rt_states\states\sc\08\ARSC0809.dbc
Reading file: C:\Users\175 MX